In [24]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import re, glob
import os, sys
from scipy import stats

# Spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from config import clientID, clientSEC


In [25]:
# Spotify API info
client_credentials_manager = SpotifyClientCredentials(client_id=clientID,client_secret=clientSEC)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [26]:
# Time Setup  (will loop later)
starting_month = 4
ending_month = 4
starting_year = 2019
ending_year = 2021
years = ending_year - starting_year

# Define country with an ISO code
country = "IT" # Italy

"""
['AD', 'AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'EC', 'SV', 'EE', 'FI'
, 'FR','DE', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'ID', 'IE', 'IT', 'JP', 'LV', 'LI', 'LT', 'LU', 'MY', 'MT', 'MX', 'MC'
, 'NL', 'NZ','NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'ES', 'SK', 'SE', 'CH', 'TW', 'TR', 'GB', 'US', 'UY']
"""

"\n['AD', 'AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'EC', 'SV', 'EE', 'FI'\n, 'FR','DE', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'ID', 'IE', 'IT', 'JP', 'LV', 'LI', 'LT', 'LU', 'MY', 'MT', 'MX', 'MC'\n, 'NL', 'NZ','NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'ES', 'SK', 'SE', 'CH', 'TW', 'TR', 'GB', 'US', 'UY']\n"

In [27]:
# Initial DataSet
df = pd.DataFrame([["latin trap", 0]], columns=["genre", "total popularity"])

In [30]:
# Variables
# Sample Size per Loop
# s_size = 50

# Script
genres_within = []
for l in range(years):
    for i in range(1): # This loop gets a sample of 50 songs per loop
        track_results = sp.search(market=country, q='year:' + str(starting_year + l), type='track', limit=50,offset=i)
        for track in track_results['tracks']['items']:

            # Make sure the result is within the timeline
            album = sp.album(track["album"]["external_urls"]["spotify"])
            try:
                if int(album["release_date"].split("-")[1]) < starting_month and int(album["release_date"].split("-")[0]) == starting_year:
                    continue
                if int(album["release_date"].split("-")[1]) > ending_month and int(album["release_date"].split("-")[0]) == ending_year:
                    continue
            except IndexError:
                continue # sometimes there are no month dates


            # get genre from artist, cant get the genre from track because spotify does not have that data
            artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
            for genre in artist["genres"]:
                if len(df.loc[df["genre"] == genre]) == 0:
                    df = pd.concat(
                        [pd.DataFrame([[genre, track["popularity"]]], columns=["genre", "total popularity"]), df],
                        ignore_index=True) # if new genre append to dataset
                df.loc[df["genre"] == genre, ["total popularity"]] += track["popularity"] # add popularity to count
                
df.to_csv("genre_data.csv") # Save to CSV
print(df.loc[df["total popularity"] > max(df["total popularity"])]) # Print most popular genre

df.head()

Empty DataFrame
Columns: [genre, total popularity]
Index: []


,genre,total popularity
0,operetta,156
1,late romantic era,156
2,classical,156
3,disco house,316
4,deep house,316
